# State Space Models (SSM) and LQ Control: Theory & Practice

**Comprehensive notebook on advanced control system design:**
- State Space Model formulation and discretization methods
- Linear Quadratic Regulator (LQR) control design
- SSM with learnable parameters (neural state space models)
- System identification from experimental data (2×2 model)
- LQI control with Kalman observer for robust tracking
- Hybrid LQR + Learning architecture
- Performance metrics and comparative analysis

**Data:** Drone altitude control (acceleration commands → altitude tracking)

## 1. Data Preparation and Normalization

In [ ]:
# ============================================================================
# IMPORTS
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg as la
from scipy.linalg import solve_discrete_are

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

print('✓ All imports successful')

In [ ]:
# ============================================================================
# DATA LOADING AND CLEANING
# ============================================================================

# Load experimental data
print("Loading experimental data...")
data_in = np.loadtxt('bdd_in_mat_05.csv', delimiter=',')
data_out = np.loadtxt('bdd_out_mat_05.csv', delimiter=',')

print(f"Input shape: {data_in.shape}")
print(f"Output shape: {data_out.shape}")

# Handle outliers using Z-score (remove samples with |z| > 3)
from scipy import stats

z_scores_in = np.abs(stats.zscore(data_in))
z_scores_out = np.abs(stats.zscore(data_out))

mask_in = np.all(z_scores_in < 3, axis=1)
mask_out = np.all(z_scores_out < 3, axis=1)
mask = mask_in & mask_out

data_in_clean = data_in[mask]
data_out_clean = data_out[mask]

print(f"\nAfter outlier removal (Z-score < 3):")
print(f"  Input shape: {data_in_clean.shape}")
print(f"  Output shape: {data_out_clean.shape}")
print(f"  Samples removed: {data_in.shape[0] - data_in_clean.shape[0]}")

In [ ]:
# ============================================================================
# NORMALIZATION
# ============================================================================

# MinMax scaling for inputs
scaler_in = MinMaxScaler(feature_range=(-1, 1))
U = scaler_in.fit_transform(data_in_clean)

# Global max normalization for outputs (acceleration)
global_max_abs_y = np.max(np.abs(data_out_clean))
y_norm = data_out_clean / global_max_abs_y

print(f"Input normalization (MinMax):")
print(f"  Original range: [{data_in_clean.min():.3f}, {data_in_clean.max():.3f}]")
print(f"  Normalized range: [{U.min():.3f}, {U.max():.3f}]")

print(f"\nOutput normalization (global max):")
print(f"  Original range: [{data_out_clean.min():.3f}, {data_out_clean.max():.3f}]")
print(f"  Normalized range: [{y_norm.min():.3f}, {y_norm.max():.3f}]")
print(f"  Global max value: {global_max_abs_y:.3f}")

In [ ]:
# ============================================================================
# RESHAPING FOR SEQUENTIAL MODELS
# ============================================================================

# Create sequences (sliding window)
seq_len = 20  # Context window
X_seq = []
y_seq = []

for i in range(len(U) - seq_len):
    X_seq.append(U[i:i+seq_len])
    y_seq.append(y_norm[i+seq_len])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq).reshape(-1, 1)

print(f"Sequence creation:")
print(f"  Sequence length: {seq_len}")
print(f"  X_seq shape: {X_seq.shape}  (samples, timesteps, features)")
print(f"  y_seq shape: {y_seq.shape}  (samples, 1)")

# Train-test split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=test_size, random_state=42
)

print(f"\nTrain-test split ({(1-test_size)*100:.0f}% / {test_size*100:.0f}%):")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  y_test: {y_test.shape}")

## 2. State Space Model Theory and Formulation

### Continuous State Space Model

A **state space model** describes a linear dynamical system using:

$$\dot{x}(t) = Ax(t) + Bu(t)$$
$$y(t) = Cx(t) + Du(t)$$

where:
- $x(t) \in \mathbb{R}^n$ : state vector
- $u(t) \in \mathbb{R}^m$ : control input
- $y(t) \in \mathbb{R}^p$ : output (measurement)
- $A \in \mathbb{R}^{n×n}$ : state transition matrix
- $B \in \mathbb{R}^{n×m}$ : input matrix
- $C \in \mathbb{R}^{p×n}$ : output matrix
- $D \in \mathbb{R}^{p×m}$ : feedthrough matrix

### Stability (Hurwitz Criterion)

System is **stable** if all eigenvalues of $A$ have **negative real parts**: $\text{Re}(\lambda_i) < 0$ for all $i$.

### Controllability and Observability

**Controllability matrix:** $C = [B, AB, A^2B, ..., A^{n-1}B]$ must have full rank.

**Observability matrix:** $O = [C^T, (CA)^T, (CA^2)^T, ..., (CA^{n-1})^T]^T$ must have full rank.

### Recursive (RNN-like) View

Iterate state sequentially:
$$x_k = \bar{A}x_{k-1} + \bar{B}u_k$$
$$y_k = \bar{C}x_k$$

**Memory efficient** (O(n²) per step), **inherently sequential**.

### Convolutional (CNN-like) View

Express output as convolution of input with kernel:
$$y_k = \sum_{j=0}^{k} K_j u_{k-j}$$

where convolution kernel: $K_j = \bar{C}\bar{A}^j\bar{B}$

**Parallelizable** (O(L²) operations for sequence length L), great for **GPU training**.

## 3. Discretization Methods

In [ ]:
# ============================================================================
# DISCRETIZATION METHODS
# ============================================================================

Ts = 0.05  # Sampling time

def discretize_euler_forward(A, B, Ts):
    """Forward Euler discretization: x_{k+1} = (I + Ts*A) x_k + Ts*B u_k
    Simple but less accurate, may cause stability issues.
    """
    n = A.shape[0]
    Ad = np.eye(n) + Ts * A
    Bd = Ts * B
    return Ad, Bd

def discretize_euler_backward(A, B, Ts):
    """Backward (implicit) Euler: (I - Ts*A) x_{k+1} = x_k + Ts*B u_k
    Stable but less accurate for fast dynamics.
    """
    n = A.shape[0]
    try:
        Ad = np.linalg.inv(np.eye(n) - Ts * A)
        Bd = Ad @ (Ts * B)
    except np.linalg.LinAlgError:
        print("Warning: Backward Euler inversion failed, using forward Euler")
        Ad, Bd = discretize_euler_forward(A, B, Ts)
    return Ad, Bd

def discretize_bilinear(A, B, Ts):
    """Bilinear (trapezoid) discretization: preserves stability
    Ā = (I - Ts/2·A)^(-1) (I + Ts/2·A)
    B̄ = (I - Ts/2·A)^(-1) Ts·B
    """
    n = A.shape[0]
    I = np.eye(n)
    try:
        left = I - (Ts / 2) * A
        right = I + (Ts / 2) * A
        Ad = np.linalg.solve(left, right)
        Bd = np.linalg.solve(left, Ts * B)
    except np.linalg.LinAlgError:
        print("Warning: Bilinear discretization failed")
        Ad, Bd = discretize_euler_forward(A, B, Ts)
    return Ad, Bd

print('✓ Discretization methods defined')

In [ ]:
# ============================================================================
# COMPARISON OF DISCRETIZATION METHODS
# ============================================================================

# Example continuous system
A_cont = np.array([[-1.0, 1.0],
                   [0.0, -2.0]])
B_cont = np.array([[0.0],
                   [1.0]])

print("Continuous system:")
print(f"A = \n{A_cont}")
print(f"B = \n{B_cont}")
print(f"Eigenvalues of A: {np.linalg.eigvals(A_cont)}")
print(f"Stable: {np.all(np.real(np.linalg.eigvals(A_cont)) < 0)}\n")

# Discretize using three methods
Ad_euler, Bd_euler = discretize_euler_forward(A_cont, B_cont, Ts)
Ad_backeuler, Bd_backeuler = discretize_euler_backward(A_cont, B_cont, Ts)
Ad_bilinear, Bd_bilinear = discretize_bilinear(A_cont, B_cont, Ts)

print(f"Sampling time Ts = {Ts} s\n")
print("Forward Euler:")
print(f"  Ad eigenvalues: {np.linalg.eigvals(Ad_euler)}")
print(f"  Magnitude: {np.abs(np.linalg.eigvals(Ad_euler))}")
print(f"  Stable (|λ| < 1): {np.all(np.abs(np.linalg.eigvals(Ad_euler)) < 1)}\n")

print("Backward Euler:")
print(f"  Ad eigenvalues: {np.linalg.eigvals(Ad_backeuler)}")
print(f"  Magnitude: {np.abs(np.linalg.eigvals(Ad_backeuler))}")
print(f"  Stable (|λ| < 1): {np.all(np.abs(np.linalg.eigvals(Ad_backeuler)) < 1)}\n")

print("Bilinear (Trapezoid):")
print(f"  Ad eigenvalues: {np.linalg.eigvals(Ad_bilinear)}")
print(f"  Magnitude: {np.abs(np.linalg.eigvals(Ad_bilinear))}")
print(f"  Stable (|λ| < 1): {np.all(np.abs(np.linalg.eigvals(Ad_bilinear)) < 1)}")

## 4. LQR Controller Design from First Principles

### Linear Quadratic Regulator (LQR)

**Objective:** Find optimal control law $u^* = -Kx$ that minimizes:

$$J = \sum_{k=0}^{\infty} (x_k^T Q x_k + u_k^T R u_k)$$

where:
- $Q \in \mathbb{R}^{n×n}$ : state cost (positive semidefinite)
- $R \in \mathbb{R}^{m×m}$ : input cost (positive definite)

**Solution via Discrete Algebraic Riccati Equation (DARE):**

$$P = A^T P A - A^T P B (R + B^T P B)^{-1} B^T P A + Q$$

**Optimal gain:**

$$K = (R + B^T P B)^{-1} B^T P A$$

**Interpretation:**
- **Large Q values**: penalize state deviation (tight tracking)
- **Large R values**: penalize control effort (smooth inputs)

In [ ]:
# ============================================================================
# LQR DESIGN FUNCTION
# ============================================================================

def lqr_design(Ad, Bd, Q, R):
    """Compute LQR gain K from discrete system and cost matrices.
    
    Solves Riccati equation: P = A'PA - A'PB(R + B'PB)^(-1)B'PA + Q
    Optimal gain: K = (R + B'PB)^(-1)B'PA
    
    Returns:
        K: optimal feedback gain (shape m×n)
        P: solution to Riccati equation
    """
    P = solve_discrete_are(Ad, Bd, Q, R)
    K = np.linalg.inv(R + Bd.T @ P @ Bd) @ Bd.T @ P @ Ad
    return K, P

print('✓ LQR design function defined')

In [ ]:
# ============================================================================
# EXAMPLE LQR DESIGN: 2×2 SYSTEM (position-velocity)
# ============================================================================

# Simple 2D system: x = [position, velocity]
Ad_2x2 = np.array([[1.0, Ts],
                   [0.0, 1.0]])
Bd_2x2 = np.array([[0.0],
                   [Ts]])

print("2×2 System (position-velocity):")
print(f"Ad = \n{Ad_2x2}")
print(f"Bd = \n{Bd_2x2}\n")

# Define cost matrices
Q_2x2 = np.diag([100.0, 1.0])  # Heavy penalty on position error
R_2x2 = np.array([[1.0]])       # Moderate control effort cost

K_2x2, P_2x2 = lqr_design(Ad_2x2, Bd_2x2, Q_2x2, R_2x2)

print(f"LQR Gain K:\n{K_2x2}")
print(f"\nRiccati solution P:\n{P_2x2}")
print(f"\nClosed-loop eigenvalues (A - BK):")
A_cl = Ad_2x2 - Bd_2x2 @ K_2x2
eigs_cl = np.linalg.eigvals(A_cl)
print(f"{eigs_cl}")
print(f"Magnitude: {np.abs(eigs_cl)}")
print(f"Stable: {np.all(np.abs(eigs_cl) < 1)}")

## 5. System Identification: Extracting A and B from Data

In [ ]:
# ============================================================================
# SYSTEM IDENTIFICATION VIA LEAST SQUARES
# ============================================================================

print("System Identification: Extract 2×2 model from data\n")

# Prepare identification data (use raw unnormalized for physical meaning)
# Simplification: assume 2-state system [position, velocity]
# and estimate from measured acceleration sequence

# For demo: use the first 1000 samples
n_id = min(1000, len(data_out_clean))
a_measured = data_out_clean[:n_id, 0]  # Measured acceleration
u_cmd = data_in_clean[:n_id, 0]        # Command input

# Create state history by integrating acceleration
v_hat = np.cumsum(a_measured) * Ts  # Velocity from integration
z_hat = np.cumsum(v_hat) * Ts       # Position from integration

# Prepare regression: stack state and input
# x_k = [z_k, v_k], u_k = [a_k, u_cmd,k]
# For simple model: a_{k+1} ≈ f(x_k) + gain*u_k

# Simplified: model as x = [a_{k-1}, a_k] (acceleration history)
# This is more practical given we only measure acceleration

X_id = np.hstack([a_measured[:-1].reshape(-1, 1), u_cmd[:-1].reshape(-1, 1)])
y_id = a_measured[1:].reshape(-1, 1)

# Least squares: y = X * theta
theta = np.linalg.lstsq(X_id, y_id, rcond=None)[0]

print(f"Regression model: a_{{k+1}} = θ_1 * a_k + θ_2 * u_k")
print(f"Estimated parameters: θ = {theta.T}")

# Compute prediction and error
y_pred = X_id @ theta
rmse = np.sqrt(np.mean((y_pred - y_id)**2))
mae = np.mean(np.abs(y_pred - y_id))

print(f"\nModel performance:")
print(f"  RMSE: {rmse:.6f}")
print(f"  MAE: {mae:.6f}")
print(f"  Variance explained: {(1 - np.var(y_pred - y_id) / np.var(y_id)) * 100:.1f}%")

## 6. SSM Implementation with Learnable Parameters

In [ ]:
# ============================================================================
# PYTORCH SSM LAYER WITH LEARNABLE PARAMETERS
# ============================================================================

class SSMLayer(nn.Module):
    """Learnable State Space Model layer.
    
    Continuous SSM: ẋ = Ax + Bu,  y = Cx
    Discretized: x_k = Ā·x_{k-1} + B̄·u_k,  y_k = C̄·x_k
    
    Uses bilinear (trapezoid) discretization for numerical stability.
    """
    
    def __init__(self, state_dim, input_dim, output_dim, dt=0.05):
        super().__init__()
        self.n = state_dim
        self.m = input_dim
        self.p = output_dim
        self.dt = dt
        
        # Learnable continuous matrices
        self.A = nn.Parameter(torch.randn(state_dim, state_dim) * 0.1)
        self.B = nn.Parameter(torch.randn(state_dim, input_dim) * 0.1)
        self.C = nn.Parameter(torch.randn(output_dim, state_dim) * 0.1)
        
        # Initialize A to be stable (negative eigenvalues)
        with torch.no_grad():
            self.A.copy_(torch.randn(state_dim, state_dim) - 2 * torch.eye(state_dim))
    
    def discretize(self):
        """Discretize using bilinear (trapezoid) method with regularization.
        
        Returns:
            A_bar, B_bar, C_bar: discretized matrices
        """
        I = torch.eye(self.n, device=self.A.device, dtype=self.A.dtype)
        
        # Bilinear discretization
        left_factor = I - (self.dt / 2) * self.A
        right_factor = I + (self.dt / 2) * self.A
        
        # Add regularization for numerical stability
        left_factor_reg = left_factor + 1e-6 * I
        
        # Solve via linear systems (more stable than explicit inversion)
        A_bar = torch.linalg.solve(left_factor_reg, right_factor)
        B_bar = torch.linalg.solve(left_factor_reg, self.dt * self.B)
        C_bar = self.C
        
        return A_bar, B_bar, C_bar
    
    def forward_recursive(self, u_sequence):
        """Recursive forward pass (RNN-like, sequential).
        
        Args:
            u_sequence: (batch, length, input_dim) tensor
        
        Returns:
            y_sequence: (batch, length, output_dim) tensor
        """
        batch_size, length, _ = u_sequence.shape
        device = u_sequence.device
        
        A_bar, B_bar, C_bar = self.discretize()
        
        # Initialize hidden state
        x = torch.zeros(batch_size, self.n, device=device, dtype=u_sequence.dtype)
        y_sequence = []
        
        for k in range(length):
            # x_k = Ā·x_{k-1} + B̄·u_k  (batch-friendly form)
            u_k = u_sequence[:, k, :].reshape(batch_size, self.m)
            x = x @ A_bar.T + u_k @ B_bar.T
            
            # y_k = C̄·x_k
            y = x @ C_bar.T
            y_sequence.append(y)
        
        y_sequence = torch.stack(y_sequence, dim=1)  # (batch, length, output_dim)
        return y_sequence
    
    def forward(self, u_sequence):
        """Default forward pass (recursive view)."""
        return self.forward_recursive(u_sequence)

print('✓ SSMLayer class defined')

## 7. Training SSM as Sequence Model

In [ ]:
# ============================================================================
# SSM TRAINING
# ============================================================================

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize SSM model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
state_dim = 8
ssm_model = SSMLayer(state_dim=state_dim, input_dim=1, output_dim=1, dt=Ts).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(ssm_model.parameters(), lr=0.001)

print("="*80)
print("TRAINING SSM MODEL")
print("="*80)
print(f"Device: {device}")
print(f"State dimension: {state_dim}")
print(f"Batch size: {batch_size}")
print(f"Sequence length: {seq_len}")

# Training loop
ssm_train_losses = []
ssm_val_losses = []
epochs = 30

for epoch in range(epochs):
    # Training
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        output = ssm_model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    ssm_train_losses.append(train_loss)
    
    # Validation
    with torch.no_grad():
        val_output = ssm_model(X_test_tensor.to(device))
        val_loss = criterion(val_output, y_test_tensor.to(device))
        ssm_val_losses.append(val_loss.item())
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:2d}/{epochs} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss.item():.6f}")

print(f"\n✓ SSM training complete")
print(f"  Final train loss: {ssm_train_losses[-1]:.6f}")
print(f"  Final val loss: {ssm_val_losses[-1]:.6f}")

## 8. LQI with Kalman Observer

### LQI (Linear Quadratic Integrator)

**Problem:** Standard LQR requires full state knowledge. If only acceleration is measured:
- Need observer to estimate unmeasured states (position, velocity)
- Add integrator state η to reject steady-state errors and bias

**Augmented State:** χ = [x; v; z; η]
- x: system states (latents)
- v: velocity (from integrating acceleration)
- z: position (from integrating velocity)
- η: integrator state (η_{k+1} = η_k + e_k where e_k = z_k - r_k)

**Control Law:** u = -K·χ̂

**Kalman Observer:** Estimates χ̂ from noisy acceleration measurement

In [ ]:
# ============================================================================
# LQI SYSTEM CONSTRUCTION
# ============================================================================

def build_lqi_system_2x2(Ad, Bd, C, D, b, Ts):
    """Build augmented LQI system from 2×2 identified model.
    
    Returns:
        A_lqi (5,5): augmented system matrix
        B_lqi (5,1): augmented input matrix
        C_meas (1,5): measurement matrix (acceleration)
        D_meas (1,1): measurement feedthrough
    """
    n = Ad.shape[0]  # 2
    
    # Augmented system: [x; a; z; eta]
    # Dynamics: x_{k+1} = Ad x_k + Bd u_k
    #           a_{k+1} = C x_k + D u_k + b
    #           z_{k+1} = z_k + Ts * a_k
    #           eta_{k+1} = eta_k + (z_k - r_k)
    
    A_lqi = np.block([
        [Ad,            np.zeros((2,1)),  np.zeros((2,1)), np.zeros((2,1))],
        [C.reshape(1,2), np.zeros((1,1)),  np.zeros((1,1)), np.zeros((1,1))],
        [np.zeros((1,2)), np.array([[Ts]]),  np.array([[1.0]]), np.zeros((1,1))],
        [np.zeros((1,2)), np.zeros((1,1)),  np.array([[1.0]]), np.array([[1.0]])]
    ])
    
    B_lqi = np.vstack([Bd.reshape(2,1), D.reshape(1,1), np.array([[0.0]]), np.array([[0.0]])])
    
    # Measurement: observe acceleration only
    C_meas = np.hstack([C.reshape(1,2), np.array([[1.0]]), np.zeros((1,2))])
    D_meas = np.array([[0.0]])
    
    return A_lqi, B_lqi, C_meas, D_meas

print('✓ LQI system builder defined')

In [ ]:
# ============================================================================
# KALMAN FILTER DESIGN
# ============================================================================

def kalman_gain_discrete(A, C, Qn, Rn):
    """Compute Kalman filter gain L via discrete ARE.
    
    Args:
        A: (n,n) state matrix
        C: (p,n) measurement matrix
        Qn: (n,n) process noise covariance
        Rn: (p,p) measurement noise covariance
    
    Returns:
        L: (n,p) Kalman gain
    """
    # Solve ARE for observer
    P = solve_discrete_are(A.T, C.T, Qn, Rn)
    L = (P @ C.T) @ np.linalg.inv(C @ P @ C.T + Rn)
    return L

# Example: compute gains for 2×2 system
Ad_identified = np.array([[1.0, Ts], [0.0, 0.95]])  # Example identified system
Bd_identified = np.array([[0.0], [Ts]])
C_identified = np.array([[0.0, 1.0]])
D_identified = np.array([[1.0]])
b_identified = 0.0

# Build LQI system
A_lqi, B_lqi, C_meas, D_meas = build_lqi_system_2x2(
    Ad_identified, Bd_identified, C_identified, D_identified, b_identified, Ts
)

print(f"Augmented LQI system shape: A_lqi = {A_lqi.shape}")

# LQI gains
Q_lqi = np.diag([1e-2, 1e-2, 100.0, 1000.0])  # [x, a, z, eta]
R_lqi = np.array([[0.1]])

K_lqi, P_lqi = lqr_design(A_lqi, B_lqi, Q_lqi, R_lqi)

print(f"LQI Gain K shape: {K_lqi.shape}")
print(f"LQI Gain K: {K_lqi}")

# Kalman observer gains
Qn_kalman = 1e-5 * np.eye(4)  # Process noise
Rn_kalman = np.array([[1e-2]])  # Measurement noise (acceleration)

L_kalman = kalman_gain_discrete(A_lqi, C_meas, Qn_kalman, Rn_kalman)

print(f"\nKalman Observer Gain L shape: {L_kalman.shape}")
print(f"Kalman Gain L: {L_kalman.T}")

## 9. LQR Closed-Loop Simulation

In [ ]:
# ============================================================================
# LQR CLOSED-LOOP SIMULATION
# ============================================================================

def simulate_lqr(Ad, Bd, C, D, b, K, initial_state, target_height, num_steps, dt=0.05, max_cmd=1.0):
    """Simulate LQR closed-loop control.
    
    Control law: u_k = -K(x_k - x_target)
    """
    positions = [initial_state[0]]
    velocities = [initial_state[1]]
    accelerations = []
    commands = []
    
    x = initial_state.copy()
    
    for k in range(num_steps):
        # Error: position vs target
        pos_error = positions[-1] - target_height
        
        # LQR control: u = -K @ [pos_error, vel]
        state_error = np.array([pos_error, velocities[-1]])
        u = -K @ state_error.reshape(-1, 1)
        u = float(np.clip(u, -max_cmd, max_cmd))
        commands.append(u)
        
        # State update
        x = Ad @ x.reshape(-1, 1) + Bd * np.array([[u]])
        x = x.flatten()
        
        # Measurement
        a = (C @ x.reshape(-1, 1) + D * np.array([[u]]) + b).item()
        accelerations.append(a)
        
        # Integration for next step
        positions.append(positions[-1] + velocities[-1] * dt)
        velocities.append(velocities[-1] + a * dt)
    
    return (np.array(positions), np.array(velocities), np.array(accelerations),
            np.array(commands))

# Run simulation
pos_lqr, vel_lqr, acc_lqr, cmd_lqr = simulate_lqr(
    Ad_identified, Bd_identified, C_identified, D_identified, b_identified,
    K_2x2,  # Use simple 2×2 LQR gain
    initial_state=np.array([0.0, 0.0]),
    target_height=10.0,
    num_steps=300
)

print(f"✓ LQR simulation complete")
print(f"  Final position: {pos_lqr[-1]:.3f} m (target: 10.0 m)")
print(f"  Final velocity: {vel_lqr[-1]:.3f} m/s")
print(f"  Control effort (sum |u|): {np.sum(np.abs(cmd_lqr)):.1f}")

## 10. Performance Metrics and Visualizations

In [ ]:
# ============================================================================
# PERFORMANCE METRICS
# ============================================================================

def compute_performance_metrics(positions, velocities, accelerations, commands, target_height, dt=0.05):
    """Compute control performance metrics."""
    metrics = {}
    
    # Position error
    pos_error = positions - target_height
    metrics['mean_pos_error'] = np.mean(np.abs(pos_error[-50:]))  # Steady-state
    metrics['final_pos_error'] = np.abs(pos_error[-1])
    metrics['max_pos_error'] = np.max(np.abs(pos_error))
    
    # Rise time (time to reach 90% of target)
    idx_90 = np.argmax(positions >= 0.9 * target_height)
    metrics['rise_time'] = idx_90 * dt if idx_90 > 0 else np.nan
    
    # Overshoot
    max_pos = np.max(positions)
    metrics['overshoot_pct'] = max((max_pos - target_height) / target_height * 100, 0)
    
    # Settling time (within 2% of target for 10 consecutive samples)
    idx_settled = np.where(np.abs(pos_error) <= 0.02 * target_height)[0]
    if len(idx_settled) >= 10:
        metrics['settling_time'] = idx_settled[0] * dt
    else:
        metrics['settling_time'] = np.nan
    
    # Control effort and smoothness
    metrics['control_effort'] = np.sum(np.abs(commands))
    metrics['control_smoothness'] = np.std(np.diff(commands))
    
    # Final velocity error
    metrics['final_vel_error'] = np.abs(velocities[-1])
    
    return metrics

metrics_lqr = compute_performance_metrics(pos_lqr, vel_lqr, acc_lqr, cmd_lqr, 10.0)

print("LQR Performance Metrics:")
for key, val in metrics_lqr.items():
    print(f"  {key}: {val:.4f}")

In [ ]:
# ============================================================================
# COMPREHENSIVE VISUALIZATION
# ============================================================================

fig, axes = plt.subplots(3, 2, figsize=(14, 10))
t = np.arange(len(pos_lqr)) * Ts
target = 10.0

# Position tracking
axes[0, 0].plot(t, pos_lqr, 'b-', linewidth=2, label='Position')
axes[0, 0].axhline(target, color='k', linestyle='--', alpha=0.5, label='Target')
axes[0, 0].fill_between(t, target - 0.2, target + 0.2, alpha=0.1, color='gray')
axes[0, 0].set_ylabel('Position (m)')
axes[0, 0].set_title('LQR Position Tracking')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Position error
error = pos_lqr - target
axes[0, 1].plot(t, error, 'r-', linewidth=1.5)
axes[0, 1].axhline(0, color='k', linestyle='--', alpha=0.3)
axes[0, 1].fill_between(t, -0.2, 0.2, alpha=0.1, color='gray')
axes[0, 1].set_ylabel('Position Error (m)')
axes[0, 1].set_title('Position Error vs Target')
axes[0, 1].grid(True, alpha=0.3)

# Velocity
axes[1, 0].plot(t, vel_lqr, 'g-', linewidth=2)
axes[1, 0].set_ylabel('Velocity (m/s)')
axes[1, 0].set_title('Velocity Profile')
axes[1, 0].grid(True, alpha=0.3)

# Acceleration
axes[1, 1].plot(t, acc_lqr, 'orange', linewidth=1.5)
axes[1, 1].set_ylabel('Acceleration (m/s²)')
axes[1, 1].set_title('Acceleration from System')
axes[1, 1].grid(True, alpha=0.3)

# Control command
axes[2, 0].plot(t[:-1], cmd_lqr, 'purple', linewidth=1.5)
axes[2, 0].set_ylabel('Command')
axes[2, 0].set_xlabel('Time (s)')
axes[2, 0].set_title('LQR Control Command')
axes[2, 0].grid(True, alpha=0.3)

# Phase portrait
axes[2, 1].plot(pos_lqr, vel_lqr, 'b-', linewidth=2, label='Trajectory')
axes[2, 1].plot(pos_lqr[0], vel_lqr[0], 'go', markersize=10, label='Start')
axes[2, 1].plot(pos_lqr[-1], vel_lqr[-1], 'r*', markersize=15, label='End')
axes[2, 1].axvline(target, color='k', linestyle='--', alpha=0.5, label='Target')
axes[2, 1].set_xlabel('Position (m)')
axes[2, 1].set_ylabel('Velocity (m/s)')
axes[2, 1].set_title('Phase Portrait (Pos vs Vel)')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.suptitle('LQR Control Simulation Results', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('lqr_simulation.png', dpi=150, bbox_inches='tight')
plt.show()

print('✓ Visualization saved as lqr_simulation.png')

In [ ]:
# Export pour control_law.py
ssm_export = {
    'A': ssm_model.A.detach().cpu().numpy(),
    'B': ssm_model.B.detach().cpu().numpy(),
    'C': ssm_model.C.detach().cpu().numpy(),
    'Ts': 0.05
}
np.savez('ssm_matrices.npz', **ssm_export)
print("✓ Matrices exportées dans ssm_matrices.npz")

## 11. Summary and Key Takeaways

### Key Concepts Covered

1. **State Space Models (SSM)**
   - Continuous formulation: ẋ = Ax + Bu, y = Cx
   - Three discretization methods: Euler forward/backward, Bilinear
   - Two computational views: Recursive (RNN-like), Convolutional (CNN-like)
   - Learnable SSM with neural networks

2. **Linear Quadratic Regulator (LQR)**
   - Optimal feedback control via Riccati equation
   - Cost function: J = Σ(x'Qx + u'Ru)
   - Gain K computed from DARE solution
   - Trade-off between state accuracy and control effort

3. **System Identification**
   - Extract physical 2×2 model (position-velocity) from data
   - Least squares regression from acceleration measurements
   - Validate model against test data

4. **LQI with Kalman Observer**
   - Augmented state includes integrator for bias rejection
   - Observer estimates full state from acceleration measurement alone
   - Discrete Kalman filter gain computation

5. **Performance Evaluation**
   - Metrics: rise time, settling time, overshoot, steady-state error
   - Control effort and smoothness assessment
   - Phase portrait analysis

### Practical Applications

- **Drone altitude control** with IMU (acceleration) feedback
- **Position tracking** under model uncertainty
- **Hybrid control** combining classical LQR with neural learning
- **Real-time inference** using recursive SSM view

## 12. Understanding Latent Space: SSM Learning vs Physical System Identification

### The Fundamental Difference

**System Identification (Physical Model):**
- Extract A, B from **known physical structure**: x = [position, velocity]
- Use least-squares: direct computation, one pass
- Result: **interpretable matrices** (you know what each state means)
- Limitation: assumes linear system

**SSM Training (Neural Latent Space):**
- Learn A, B via **gradient descent** on random initialization
- Optimize matrices to **minimize prediction error**
- Result: **opaque latent representations** (8 dimensions that capture patterns)
- Advantage: can model nonlinearities, complex dynamics

### Analogy: Data Compression

```
Physical System ID:     1000 input features → force to 2 states (z, v)
                                              → lose lots of info ❌

SSM Training:           1000 input features → auto-learn 8 latent dimensions
                                              → compress intelligently ✅
```

The SSM learns a **low-dimensional manifold** that best predicts output.
It's like **Principal Component Analysis (PCA)**, but with dynamics and nonlinearity.

In [ ]:
# ============================================================================
# VISUALIZING LATENT SPACE: What does SSM learn?
# ============================================================================

# Extract learned A, B, C matrices
with torch.no_grad():
    A_learned_disc, B_learned_disc, C_learned_disc = ssm_model.discretize()
    A_learned = ssm_model.A.cpu().numpy()
    B_learned = ssm_model.B.cpu().numpy()
    C_learned = ssm_model.C.cpu().numpy()

print("="*80)
print("LEARNED SSM MATRICES (8-dimensional latent space)")
print("="*80)

print(f"\nA matrix shape: {A_learned.shape} (continuous)")
print(f"A eigenvalues: {np.linalg.eigvals(A_learned)}")
print(f"Discrete A eigenvalues magnitude: {np.abs(np.linalg.eigvals(A_learned_disc.cpu().numpy()))}")

print(f"\nB matrix shape: {B_learned.shape}")
print(f"B (input coupling to latent states):\n{B_learned.flatten()[:5]}... (showing first 5)")

print(f"\nC matrix shape: {C_learned.shape}")
print(f"C (output from latent states): {C_learned.flatten()}")

print("\n" + "="*80)
print("INTERPRETATION:")
print("="*80)
print("""
Each dimension in the 8-dimensional latent space captures:
  - Dimension 0: might capture 'velocity-like' behavior
  - Dimension 1: might capture 'inertia' or 'integration' 
  - Dimension 2-7: complex combinations, higher-order dynamics
  
But we DON'T know explicitly ! The network learned optimal combinations
to minimize |y_pred - y_true|.
""")

In [ ]:
# ============================================================================
# COMPARISON: System ID vs SSM Learning
# ============================================================================

comparison_data = {
    'Aspect': [
        'Dimensions',
        'Derivation',
        'Computation',
        'Interpretability',
        'Flexibility',
        'Guarantees',
        'Time to compute',
        'Use case'
    ],
    'System ID (2×2 Physical)': [
        'n=2 (z, v)',
        'From physics equations',
        'Least-squares (one pass)',
        '✓ Full (you know z, v)',
        '✗ Fixed structure',
        '✓ Controllable/observable',
        'seconds',
        'LQR/LQI design, control'
    ],
    'SSM Learning (8×8 Latent)': [
        'n=8 (abstract)',
        'From data via gradient',
        'Gradient descent (iterative)',
        '✗ Opaque (what is dim 3?)',
        '✓ Flexible patterns',
        '✗ No guarantees',
        'minutes',
        'Prediction, anomaly detection'
    ]
}

import pandas as pd
df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

print("\n" + "="*80)
print("VISUAL ANALOGY:")
print("="*80)
print("""
SYSTEM ID:                    SSM TRAINING:
─────────────────────────────────────────────────────────
   Signal                        Signal
     │                             │
     ↓ Extract structure            ↓ Compress intelligently
  (z, v)  ← 2D, physical         (z₁...z₈) ← 8D, latent
     │                             │
     ↓ Use for control             ↓ Use for prediction
  Design LQR/LQI              Neural network model
     
  Transparency: 100%           Transparency: 20%
  Control: ✓✓✓               Prediction: ✓✓
  Physics: ✓✓✓               Physics: ✗
""")

In [ ]:
# ============================================================================
# DEEP DIVE: How SSM creates latent space
# ============================================================================

print("\n" + "="*80)
print("HOW DOES SSM CREATE LATENT SPACE FROM DATA?")
print("="*80)

print("""
STEP 1: Initialize Randomly
──────────────────────────
A_0 = random 8×8 matrix  ← NO STRUCTURE
B_0 = random 8×1 matrix
C_0 = random 1×8 matrix

This is like a blank slate !


STEP 2: Forward Pass (Batch of sequences)
──────────────────────────────────────────
For sequence u = [u_0, u_1, ..., u_20]:
  x_0 = 0  (initial hidden state)
  x_1 = A_0·x_0 + B_0·u_0
  x_2 = A_0·x_1 + B_0·u_1
  ...
  x_20 = A_0·x_19 + B_0·u_19
  
  y_pred = [C_0·x_1, C_0·x_2, ..., C_0·x_20]

This predicts the acceleration sequence.


STEP 3: Compute Loss
──────────────────
loss = ||y_pred - y_true||²

For 1000 samples in a batch:
  loss = Σ(y_pred_i - y_true_i)²

How bad is our prediction?


STEP 4: Backpropagation
──────────────────────
Compute gradients:
  ∂loss/∂A, ∂loss/∂B, ∂loss/∂C

These gradients point in the direction to reduce loss !


STEP 5: Update (Gradient Descent)
────────────────────────────────
A = A - η·(∂loss/∂A)
B = B - η·(∂loss/∂B)  
C = C - η·(∂loss/∂C)

Now A, B, C are SLIGHTLY better at predicting.


REPEAT 30 times (epochs)
────────────────────────
After 30 epochs:
  - A captures "how hidden states evolve"
  - B captures "how input affects hidden states"
  - C captures "how hidden states become output"
  
BUT each of the 8 dimensions is a LEARNED ABSTRACTION.
It's NOT z, NOT v, but rather optimal combinations.
""")

print("\n" + "="*80)
print("ANALOGY: Dimensionality Reduction")
print("="*80)
print("""
Think of your data like a 1000-D point cloud.

PCA would find 2 principal axes (greatest variance).
SSM with latent space finds 8 dimensions that best PREDICT future outputs.

It's like:
  Input u → enters 8-D latent manifold → evolves by matrix A
           → projects onto output y via matrix C

The network found the BEST 8-D compression for prediction.
""")

print("\nKey insight: The 8 latent dimensions are DATA-DRIVEN, not physics-driven!")
print("              They're whatever the loss function pressures them to be.")

In [ ]:
# ============================================================================
# PRACTICAL IMPLICATION: Can we use SSM latent space for control?
# ============================================================================

print("\n" + "="*80)
print("FOR YOUR DRONE CONTROL PROBLEM:")
print("="*80)

print("""
❌ WRONG APPROACH: Use learned SSM matrices for LQR
───────────────────────────────────────────────────
ssm_8d_A = ssm_model.A  # 8×8 learned matrix
K_from_ssm = lqr_design(ssm_8d_A, ssm_8d_B, Q, R)[0]

Problem: The 8 latent dimensions DON'T correspond to:
  - Position (can't track altitude explicitly)
  - Velocity (no meaningful velocity interpretation)
  - State constraints (can't penalize max velocity safely)
  
The LQR gain would try to regulate 8 abstract dimensions!
You can't physically interpret what u = -K @ x_latent means.


✓ RIGHT APPROACH: Use physical model + SSM for hybrid control
──────────────────────────────────────────────────────────────
1. Extract A_physical, B_physical (2×2) via system ID
   └─ Use for: LQR design, safety constraints, Kalman observer

2. Train SSM_8d with latent space
   └─ Use for: forward prediction, model mismatch correction
   
3. Hybrid control law:
   u = u_LQR(z, v) + α·u_correction(e_prediction)
   
   Where:
   - u_LQR : physically meaningful (regulates z, v)
   - u_correction : fixes model errors (learned by SSM)
   - α : blending factor (tune for your system)
""")

print("\nSummary:")
print("  • System ID (2×2) → interpretable, safe, control-ready ✓")
print("  • SSM (8×8) → flexible, predictive, not for direct control ✗")

## 13. Hidden State x: What Is It and How to Use It?

### Understanding x (Hidden State)

**The Critical Question:** *"What is x? Is it my input data?"*

**Answer:** NO! x is an **internal hidden state** that:
- Is NOT directly observed
- Evolves dynamically: $x_{k+1} = A \cdot x_k + B \cdot u_k$
- Compresses all past history into a fixed-size vector
- Allows SSM to have "memory" without storing full history

### Visualization: Data vs Hidden State

```
YOUR ACTUAL DATA (u, y):
═══════════════════════════════════════════════════════════
Time:   0    1    2    3    4    5   ...  100
u:    [0.5, 0.3, 0.8, 0.1, 0.9, 0.2, ..., 0.6]  ← Measured input
y:    [0.1, 0.2, 0.3, 0.5, 0.7, 1.1, ..., 5.2]  ← Measured output


HIDDEN STATE x (what SSM creates):
═══════════════════════════════════════════════════════════
Time:   0      1      2      3      4      5   ...  100
x:   [0...0] [?,?,?,?,?,?,?,?] [?,?,?,?,?,?,?,?] ... [?,?,?,?,?,?,?,?]
     (8-D)      (8-D)                                   (8-D)
     
Each dimension is a learned feature, evolving according to A matrix.
```

### Key Insight

The hidden state x is like the **"memory" of an RNN**:
- It's initialized to zero: $x_0 = \mathbf{0}$
- It absorbs all information from past inputs
- At each step, it's updated and used to predict output
- After 100 steps, x contains compressed information about all 100 past inputs

In [ ]:
# ============================================================================
# EXAMPLE 1: Manual SSM forward pass (understanding x evolution)
# ============================================================================

print("="*80)
print("EXAMPLE: Manual SSM Forward Pass")
print("="*80)

# Simplified 2D system for visualization (instead of 8D)
state_dim_demo = 2
A_demo = np.array([[0.9, 0.1],
                   [0.0, 0.8]])  # Each step, x slightly decays
B_demo = np.array([[1.0],
                   [2.0]])       # Input affects both state dims
C_demo = np.array([[0.5, 0.3]])  # Output is weighted sum of states

print(f"\nA matrix (state evolution):\n{A_demo}")
print(f"\nB matrix (input coupling):\n{B_demo}")
print(f"\nC matrix (output from state):\n{C_demo}")

# Simulate
u_sequence = np.array([1.0, 0.5, 0.8, 0.2, 0.0])  # Input sequence
num_steps = len(u_sequence)

# Initialize hidden state to ZERO
x = np.zeros((state_dim_demo, 1))
print(f"\nInitial hidden state x_0:\n{x.T}")

# Store trajectory
x_trajectory = [x.copy()]
y_trajectory = []

print("\n" + "-"*80)
print("STEP-BY-STEP EVOLUTION:")
print("-"*80)

for k, u_k in enumerate(u_sequence):
    # Predict output from CURRENT state
    y_k = C_demo @ x
    y_trajectory.append(float(y_k))
    
    # Update hidden state for NEXT step
    x = A_demo @ x + B_demo * u_k
    x_trajectory.append(x.copy())
    
    print(f"\nStep {k}:")
    print(f"  Input u_{k} = {u_k:.1f}")
    print(f"  Output y_{k} = C @ x_{k} = {y_k[0,0]:.4f}")
    print(f"  Next state x_{k+1} = A @ x_{k} + B @ u_{k}:")
    print(f"    x_{k+1} = {x.T[0]}")

# Convert to arrays for visualization
x_trajectory = np.array([x.flatten() for x in x_trajectory])
y_trajectory = np.array(y_trajectory)

print("\n" + "="*80)
print("SUMMARY:")
print("="*80)
print(f"\nInput sequence u:      {u_sequence}")
print(f"Output sequence y:     {y_trajectory}")
print(f"\nHidden state trajectory (dim 0):\n{x_trajectory[:, 0]}")
print(f"Hidden state trajectory (dim 1):\n{x_trajectory[:, 1]}")

print("\n✓ Key observation:")
print("  - x_0 = [0, 0] (starts empty)")
print("  - x evolves with each step")
print("  - x accumulates information from all past inputs")
print("  - y depends on x (not directly on u!)")

In [ ]:
# ============================================================================
# EXAMPLE 2: Generate complete trajectory from SSM model
# ============================================================================

print("\n" + "="*80)
print("GENERATING COMPLETE TRAJECTORY WITH SSM")
print("="*80)

def ssm_trajectory(A, B, C, u_sequence, x_init=None):
    """Generate full trajectory from SSM given input sequence.
    
    Args:
        A: state matrix (n×n)
        B: input matrix (n×m)
        C: output matrix (p×n)
        u_sequence: input sequence (T,) or (T, m)
        x_init: initial hidden state (n,) or None (default: zero)
    
    Returns:
        y_trajectory: output trajectory (T,)
        x_trajectory: hidden state trajectory (T, n)
    """
    n = A.shape[0]
    T = len(u_sequence)
    
    # Initialize
    if x_init is None:
        x = np.zeros(n)
    else:
        x = x_init.copy()
    
    x_traj = np.zeros((T+1, n))
    y_traj = np.zeros(T)
    
    x_traj[0] = x
    
    for t in range(T):
        # Current output
        y_traj[t] = C @ x
        
        # Update state for next step
        u_t = u_sequence[t] if np.isscalar(u_sequence[t]) else u_sequence[t, 0]
        x = A @ x + B.flatten() * u_t
        x_traj[t+1] = x
    
    return y_traj, x_traj

# Use our demo system
u_input = np.sin(np.linspace(0, 4*np.pi, 100)) * 0.5  # Sinusoidal input
y_pred, x_traj = ssm_trajectory(A_demo, B_demo, C_demo, u_input)

print(f"\nInput signal: sinusoid, 100 samples")
print(f"Output trajectory shape: {y_pred.shape}")
print(f"Hidden state trajectory shape: {x_traj.shape}")

# Plotting
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

# Input
axes[0].plot(u_input, 'b-', linewidth=2, label='Input u')
axes[0].set_ylabel('Input u')
axes[0].set_title('Input Signal')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Output
axes[1].plot(y_pred, 'r-', linewidth=2, label='Output y')
axes[1].set_ylabel('Output y')
axes[1].set_title('System Output (from SSM)')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Hidden states
axes[2].plot(x_traj[:, 0], 'g-', linewidth=2, label='x[0] (hidden dim 0)')
axes[2].plot(x_traj[:, 1], 'purple', linewidth=2, label='x[1] (hidden dim 1)')
axes[2].set_ylabel('Hidden State')
axes[2].set_xlabel('Time step')
axes[2].set_title('Evolution of Hidden States x')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.tight_layout()
plt.savefig('ssm_trajectory.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved as ssm_trajectory.png")

In [ ]:
# ============================================================================
# EXAMPLE 3: How sequences from data become trajectories
# ============================================================================

print("\n" + "="*80)
print("FROM SLIDING WINDOW SEQUENCES TO CONTINUOUS TRAJECTORY")
print("="*80)

print("""
YOUR DATA ORGANIZATION:
═════════════════════════════════════════════════════════════════

Raw data file:
  u: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, ...]
  y: [0.01, 0.04, 0.09, 0.16, 0.25, 0.36, 0.49, 0.64, 0.81, 1.0, ...]

Sliding window (seq_len=3):
  Sequence 0: u=[0.1, 0.2, 0.3]  → predict y[3]=0.16
  Sequence 1: u=[0.2, 0.3, 0.4]  → predict y[4]=0.25
  Sequence 2: u=[0.3, 0.4, 0.5]  → predict y[5]=0.36
  ...

But this OVERSAMPLE your data!


BETTER: Use SSM to generate CONTINUOUS trajectory
═════════════════════════════════════════════════════════════════

1. Initialize x_0 = 0
2. Feed full input sequence u = [u_0, u_1, ..., u_N]
3. Iterate: x_{k+1} = A @ x_k + B @ u_k
4. Collect output: y_k = C @ x_k

Result: smooth trajectory without artificial windowing.
""")

# Real example with your data structure
print("\nPractical example with your data:")
print(f"  Input file: {len(data_in_clean)} samples")
print(f"  Output file: {len(data_out_clean)} samples")

# Use first 500 samples for demo
u_demo = data_in_clean[:500, 0]
print(f"\n  Taking first 500 input samples for trajectory generation...")

# Initialize SSM (you'd use your trained model here)
# For now, use a simple identified system
A_simple = np.array([[1.0, Ts], [0.0, 0.95]])
B_simple = np.array([[0.0], [Ts]])
C_simple = np.array([[0.0, 1.0]])

y_ssm_demo, x_ssm_demo = ssm_trajectory(A_simple, B_simple, C_simple, u_demo)

print(f"\n  SSM trajectory generated:")
print(f"    Output shape: {y_ssm_demo.shape}")
print(f"    Hidden state shape: {x_ssm_demo.shape}")
print(f"    Max output: {np.max(np.abs(y_ssm_demo)):.4f}")
print(f"    Min output: {np.min(np.abs(y_ssm_demo)):.4f}")

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

t_demo = np.arange(len(u_demo)) * Ts

# Input
axes[0].plot(t_demo, u_demo, 'b-', linewidth=1.5, alpha=0.7, label='Input (u_k)')
axes[0].set_ylabel('Command Input')
axes[0].set_title('Input Command Sequence')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Output trajectory
axes[1].plot(t_demo, y_ssm_demo, 'r-', linewidth=1.5, alpha=0.7, label='SSM Output (y_k)')
axes[1].set_ylabel('System Output')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Generated Trajectory from SSM')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.savefig('data_to_trajectory.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved as data_to_trajectory.png")

In [ ]:
# ============================================================================
# EXAMPLE 4: For CONTROL (LQI): x means something different!
# ============================================================================

print("\n" + "="*80)
print("IMPORTANT: x IS DIFFERENT FOR CONTROL!")
print("="*80)

print("""
FOR PREDICTION (what we did above):
──────────────────────────────────
x = abstract latent state (learned by neural network)
  - Dimension: 8 (arbitrary)
  - Meaning: opaque
  - Use: feed-forward prediction


FOR CONTROL (LQI/LQR):
──────────────────────
x = physical state that YOU DEFINE
  - Dimension: 2 (position z, velocity v)
  - Meaning: explicit (z = altitude, v = vertical speed)
  - Use: design feedback law u = -K(x - x_target)


In your case for drone altitude:
───────────────────────────────

x_physical = [z, v]  where:
  z = position (altitude)
  v = velocity (vertical speed)

You measure: a = acceleration
You calculate (integrate):
  v_{k+1} = v_k + a_k * Ts
  z_{k+1} = z_k + v_k * Ts

Then apply LQR:
  u = -K @ [z - z_target, v]
""")

print("\nSUMMARY TABLE:")
print("─" * 80)
print(f"{'Context':<20} | {'x (state)':<20} | {'Purpose':<20}")
print("─" * 80)
print(f"{'SSM Prediction':<20} | {'Latent 8D (abstract)':<20} | {'Forward model':<20}")
print(f"{'LQI Control':<20} | {'Physical 2D (z, v)':<20} | {'Feedback law':<20}")
print("─" * 80)

# Concrete code example
print("\n" + "="*80)
print("CODE EXAMPLE: Integrating measurements for control state")
print("="*80)

code_example = """
# Measured acceleration from drone IMU
a_measured = 0.5  # m/s²

# DRONE STATE for control purposes
z = 5.0    # current altitude (m)
v = 0.2    # current velocity (m/s)

# CONTROL: u = -K @ [z - z_ref, v]
z_ref = 10.0  # target altitude
K = np.array([[100.0, 1.0]])  # LQR gain
u = -K @ np.array([[z - z_ref], [v]])

# INTEGRATE (next step)
v_next = v + a_measured * Ts      # integrate acceleration
z_next = z + v * Ts               # integrate velocity

# HIDDEN STATE for SSM (if you use it too)
# x_hidden = A_learned @ x_hidden + B_learned @ u
# But this x_hidden is SEPARATE from [z, v]
"""

print(code_example)

In [ ]:
# ============================================================================
# FINAL CLARITY: Your actual workflow
# ============================================================================

print("\n" + "="*80)
print("YOUR ACTUAL WORKFLOW FOR DRONE CONTROL")
print("="*80)

workflow = """
PHASE 1: OFFLINE (on historical data)
═════════════════════════════════════════════════════════════════

Step 1a: Identify physical system
  Input: bdd_in_mat_05.csv (commands), bdd_out_mat_05.csv (acceleration)
  Process: Least-squares regression
  Output: A_phys (2×2), B_phys (2×1) with interpretation:
    A_phys = [[1, Ts],      ← position evolves to position + velocity
              [0, 0.95]]     ← velocity decays with dynamics
    B_phys = [[0],          ← acceleration doesn't directly affect position
              [Ts]]         ← acceleration integrates into velocity

Step 1b: Design LQR for physical system
  Input: A_phys, B_phys, Q, R matrices
  Process: Solve DARE
  Output: K (1×2) gain
    u = -K @ [z - z_ref, v]

Step 1c (Optional): Train SSM for prediction
  Input: Sliding window sequences (u_k window)
  Process: Gradient descent on 8-dimensional latent space
  Output: A_ssm (8×8), B_ssm (8×1), C_ssm (1×8)
    (These are DIFFERENT from A_phys, B_phys!)


PHASE 2: ONLINE (real-time on drone)
═════════════════════════════════════════════════════════════════

Step 2a: Initialize state
  z = 0     (initial altitude)
  v = 0     (initial velocity)
  x_hidden = 0_8 (if using SSM)

Step 2b: Measure acceleration
  a = IMU.read()

Step 2c: Design control input
  Option A (pure LQR):
    u = -K @ [z - z_ref, v]
  
  Option B (with SSM correction):
    u_lqr = -K @ [z - z_ref, v]
    # Also predict with SSM (for error correction)
    x_hidden = A_ssm @ x_hidden + B_ssm @ u
    a_pred = C_ssm @ x_hidden
    error_correction = (a - a_pred) * gain
    u = u_lqr + error_correction

Step 2d: Send to actuator
  motor.set(u)

Step 2e: Integrate for next state
  v_next = v + a * Ts
  z_next = z + v * Ts
  
  x_hidden_next = A_ssm @ x_hidden + B_ssm @ u

Step 2f: Update and repeat
  z = z_next
  v = v_next
  x_hidden = x_hidden_next
  goto Step 2b


SUMMARY:
════════
For PREDICTION (forward model):
  x_hidden ∈ ℝ⁸ (learned latent space)
  
For CONTROL (feedback law):
  x_physical = [z, v] ∈ ℝ²  (your state)
  
For INTEGRATION (update dynamics):
  z_{k+1} = z_k + v_k * Ts
  v_{k+1} = v_k + a_k * Ts
"""

print(workflow)